<a href="https://colab.research.google.com/github/dhnanjay/HuggingFace/blob/main/YT_vLLM_Vicuna13B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain #tiktoken #duckduckgo-search
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip -q install sentencepiece Xformers einops
!pip -q install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.8/155.8 kB 22.3 

In [ ]:
import os


## Load Model

In [ ]:
model ="lmsys/vicuna-13b-v1.3"

In [ ]:
from vllm import LLM, SamplingParams

In [ ]:
prompts = [
    "Hello, how are you?",
    # "The president of the United States is",
    # "The capital of France is",
    # "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [ ]:
llm = LLM(model=model)

INFO 07-07 01:07:58 llm_engine.py:60] Initializing an LLM engine with config: model='lmsys/vicuna-13b-v1.3', tokenizer='lmsys/vicuna-13b-v1.3', tokenizer_mode=auto, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)


INFO 07-07 01:13:22 llm_engine.py:131] # GPU blocks: 875, # CPU blocks: 327


In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]

Prompt: 'Hello, how are you?', Generated text: "It's great to be back here with you all again. As you know"


In [ ]:
import json
import textwrap

# def get_prompt(instruction):
#     prompt_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
#     return prompt_template.format(instruction=instruction)

system_prompt = "A chat between a curious user and an artificial intelligence assistant. \nThe assistant gives helpful, detailed, and polite answers to the user's questions."

addon_prompt = ""
# USER: What is 4x8?
# ASSISTANT:
def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template=f"{system_prompt}\n{addon_prompt} \n\nUSER: {human_prompt} \nASSISTANT: "
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text):
    prompt = get_prompt(text)
    sampling_params = SamplingParams(
                                    max_tokens=512,
                                    # do_sample=True,
                                    temperature=0.7,
                                    top_p =0.95,
                                    top_k =  50,
                                    # eos_token_id=tokenizer.eos_token_id,
                                    # pad_token_id=tokenizer.pad_token_id,
                                     )
    outputs = llm.generate([prompt], sampling_params)
    return outputs

def parse_text(output):
        generated_text = output[0].outputs[0].text
        wrapped_text = textwrap.fill(generated_text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text
        # prompt = output.prompt



In [ ]:
%%time
prompt = 'What are the differences between alpacas, vicunas and llamas?'
generated_text = generate(prompt)
final_text = parse_text(generated_text)
print(final_text)

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.03s/it]

1. Physical Characteristics: Alpacas are smaller and more delicate than llamas, with shorter necks
and legs. They have a distinctive hump on their back and a straight profile. Alpacas have a
luxurious fleece that comes in a variety of colors, including white, black, brown, and gray. Vicunas
are even smaller and more delicate than alpacas, with a longer neck and legs. They have a more
pronounced hump on their back and a more curved profile. Vicunas have a shorter, finer fleece that
comes in a range of colors, including brown, black, and white. Llamas are larger and more robust
than both alpacas and vicunas, with a longer neck and legs. They have a distinctive hump on their
back and a more pronounced profile. Llamas have a coarser fleece that comes in a variety of colors,
including white, black, brown, and gray. 2. Habitat: Alpacas are native to the high Andes mountains
in South America, primarily in Peru, Chile, and Bolivia. They are adapted to life at high altitudes
and can survive in 

In [ ]:
%%time
prompt = 'What is the capital of England?'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]

  The capital of England is London.  It is the largest city in England and the United Kingdom, and
it is also one of the largest cities in the world. London is known for its rich history, cultural
landmarks, and diverse population. It is home to many famous landmarks such as Buckingham Palace,
the Tower Bridge, and the British Museum. The city is also known for its world-class theaters,
museums, and galleries.</s>


CPU times: user 2.34 s, sys: 0 ns, total: 2.34 s
Wall time: 2.34 s


In [ ]:
%%time
prompt = 'Write an email to Sam Altman giving reasons to open source GPT-4'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.02s/it]

  Subject: Reasons to Open Source GPT-4  Dear Mr. Altman,  I hope this email finds you well. I am
writing to express my thoughts on the potential benefits of open sourcing GPT-4, the latest
iteration of the GPT (Generative Pre-trained Transformer) language model. I believe that open
sourcing this powerful AI technology can bring about numerous advantages for society, the industry,
and your company, Y Combinator. Here are some key reasons to consider:  1. Accelerated Innovation:
Open sourcing GPT-4 will allow researchers, developers, and organizations from around the world to
collaborate and build upon the technology. This can lead to rapid innovation and the development of
new applications, tools, and use cases that may not have been possible under a proprietary model. 2.
Democratization of AI: By making GPT-4 open source, you can help bridge the gap between large tech
companies and smaller organizations or individuals who may not have the resources to develop AI
technology on their ow

In [ ]:
%%time
prompt = 'Write an email to Sam Altman giving reasons to open source GPT-4'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.71s/it]

  Subject: Open Sourcing GPT-4: A Vision for a Better Future  Dear Mr. Sam Altman,  I hope this
email finds you well. I am writing to you today to discuss the potential benefits of open sourcing
GPT-4, the latest version of the GPT series of language models. As you may already know, GPT-4 has
garnered significant attention for its remarkable ability to understand and generate human-like
text. By open sourcing this technology, we can unlock a wealth of opportunities for innovation,
collaboration, and positive societal impact.  1. Foster Innovation and Collaboration  Open sourcing
GPT-4 will enable a global community of developers, researchers, and organizations to collaborate,
share ideas, and build upon the technology. This will accelerate the development of new
applications, improve the model's performance, and lead to the creation of novel solutions to
various challenges. By providing access to the source code, we can empower a diverse range of
contributors to contribute to the proje

In [ ]:
%%time
prompt = 'As an AI do you like the Simpsons? What do you know about Homer?'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.47s/it]

  As an AI, I don't have personal preferences or emotions, so I don't "like" or "dislike" anything,
including the Simpsons. However, I can provide you with information about the show and its
characters.  Homer Jay Simpson is the protagonist of the animated television series "The Simpsons,"
which first aired in 1989. He is a father, husband, and nuclear power plant employee who is known
for his yellow skin, blue hair, and distinctive d'oh! exclamation. Homer is portrayed as a loving
but irresponsible and easily distracted father, who often finds himself in bizarre situations due to
his own mistakes. He is married to Marge Simpson and has three children: Bart, Lisa, and Maggie.
Throughout the show's run, Homer has become a cultural icon and is widely recognized as one of the
most popular and enduring characters in television history. His catchphrases, such as "d'oh!" and
"mmmm... donuts," have become part of the American lexicon, and his antics have endeared him to
audiences around the w

In [ ]:
%%time
prompt = 'Tell me about Homer on the TV show the simpsons'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.44s/it]

1. "The Many Faces of Homer Simpson" 2. "Homer Simpson: The Iconic Character of The Simpsons" 3.
"Homer Simpson: The Loveable Rogue of Springfield" 4. "Homer Simpson: The D'oh-n of The Simpsons" 5.
"Homer Simpson: The Beer-Loving Patriarch of The Simpsons" 6. "Homer Simpson: The Blue-Collar Hero
of The Simpsons" 7. "Homer Simpson: The Tactless Troublemaker of The Simpsons" 8. "Homer Simpson:
The Unlikely Hero of The Simpsons" 9. "Homer Simpson: The Unforgettable Character of The Simpsons"
10. "Homer Simpson: The Loveable Oaf of The Simpsons" 11. "Homer Simpson: The Iconic TV Dad of The
Simpsons" 12. "Homer Simpson: The Funniest Character on TV" 13. "Homer Simpson: The Most
Recognizable Character on TV" 14. "Homer Simpson: The Legendary Character of The Simpsons" 15.
"Homer Simpson: The One and Only of The Simpsons"</s>


CPU times: user 7.46 s, sys: 0 ns, total: 7.46 s
Wall time: 7.44 s


In [ ]:
%%time
prompt = 'Tell me about Homer on the TV show the simpsons in depth'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.01s/it]

1. "Homer Simpson: A D'oh-cious Guide to the Greatest Dad in the World" by Matt Groening 2. "The
Simpsons and Philosophy: The D'oh! of Homer" edited by William Irwin 3. "Homer Simpson's Little Book
of Laziness" by Matt Groening 4. "The Simpsons Season 3 Episode 17 'The Simpson's 138th Episode
Spectacular': A Guide to the Best Show on Television" by Alan Sepinwall 5. "Homer Simpson's Guide to
Life: A Collection of Favourite Wisdom from the Most Wise Man in Springfield" by Matt Groening 6.
"The Simpsons: A Complete Guide to Our Favorite Family" by Tony Sánchez 7. "The Simpsons: A Cultural
History" by C. W. Marshall 8. "The Simpsons and the Philosophy of Life" by Mark D. White 9. "Homer
Simpson's Book of Lists" by Matt Groening 10. "The Simpsons: The Complete Seventeenth Season" by
Matt Groening 11. "The Simpsons: The Complete Eighteenth Season" by Matt Groening 12. "The Simpsons:
The Complete Twentieth Season" by Matt Groening 13. "The Simpsons: The Complete Twenty-First Season"
by Matt 

In [ ]:

%%time
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.79s/it]

1. Start with the given information: The cafeteria had 23 apples and they used 20 for lunch. 2.
Subtract the number used for lunch from the total apples to find the remaining apples: 23 - 20 = 3.
3. Add the number of apples bought to find the total apples after buying more: 3 + 6 = 9. 4. The
cafeteria now has 9 apples after buying more.</s>


CPU times: user 2.81 s, sys: 0 ns, total: 2.81 s
Wall time: 2.8 s


In [ ]:
%%time
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.83s/it]

1. Yes 2. A Haiku is a traditional form of Japanese poetry, consisting of three lines with 5, 7, and
5 syllables respectively. 3. Twitter has a 280 character limit per tweet. 4. Since each line of a
Haiku typically has around 10-12 syllables, it is possible to fit a Haiku within the 280 character
limit of a tweet. 5. Therefore, it is possible to write a Haiku in a single tweet.</s>


CPU times: user 2.85 s, sys: 0 ns, total: 2.85 s
Wall time: 2.84 s


In [ ]:
%%time
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.81s/it]

1. "Unlocking the Secrets of Hogwarts: A Beginner's Guide to Harry Potter" 2. "Hogwarts 101:
Mastering the Art of Wizardry with Harry Potter" 3. "The Magic of Hogwarts: A Journey Through Harry
Potter's School Years" 4. "From Gryffindor to Slytherin: The Different Houses of Hogwarts" 5. "The
Curriculum of Hogwarts: Exploring the Subjects Studied at School" 6. "The Forbidden Forest and
Beyond: Adventures in the Hogwarts Curriculum" 7. "The Triwizard Tournament: Hogwarts at its Most
Competitive" 8. "The Chamber of Secrets: Uncovering the Mysteries of Hogwarts" 9. "The Battle of
Hogwarts: A Look Back at the Final Battle" 10. "The Return to Hogwarts: A Journey Through the Post-
War Years"</s>


CPU times: user 5.83 s, sys: 0 ns, total: 5.83 s
Wall time: 5.81 s


In [ ]:
%%time
prompt = """Convert the following to JSON

name: John
age: 30
address:
street: 123 Main Street
city: San Fransisco
state: CA
zip: 94101
"""
generated_text = generate(prompt)
parse_text(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.67s/it]

  { "name": "John", "age": 30, "address": { "street": "123 Main Street", "city": "San Fransisco",
"state": "CA", "zip": "94101" } }</s>


CPU times: user 1.68 s, sys: 0 ns, total: 1.68 s
Wall time: 1.67 s


In [ ]:
%%time
prompt = """Write me a short plan for a 3 day trip to London"""
generated_text = generate(prompt)
final_text = parse_text(generated_text)
print(final_text)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.60s/it]

1. Day 1: Arrival and Exploring Central London * Morning: Arrive in London and check into your hotel
* Afternoon: Visit some of the most iconic landmarks in London, such as the Tower of London, Tower
Bridge, and the British Museum. * Evening: Enjoy a traditional British meal and take in a West End
show. 2. Day 2: London's Historic Sites and Royalty * Morning: Explore the historic city of
Westminster, including the Houses of Parliament, Big Ben, and Buckingham Palace. * Afternoon: Visit
the Churchill War Rooms and learn about the life of Winston Churchill. * Evening: Stroll along the
South Bank of the River Thames, enjoying the views and street performers. 3. Day 3: Green Spaces and
Art * Morning: Visit the Natural History Museum and the Science Museum. * Afternoon: Explore Hyde
Park, one of London's largest and most famous parks, and visit the Serpentine Gallery. * Evening:
Have dinner in Chinatown and take a walk through Soho, one of London's most vibrant and eclectic
neighborhoods.  

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM



In [ ]:

tokenizer = AutoTokenizer.from_pretrained("TheBloke/wizard-vicuna-13B-HF")

model = AutoModelForCausalLM.from_pretrained("TheBloke/wizard-vicuna-13B-HF",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_8bit=True)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Paris
B: Marseille
C: Lyon
D: Toulouse
I have been looking for this for a while.
I was told that the capital of France is Paris.
But I'm not sure if this is true.
I was also told that the capital of France is Lyon.
But I'm not sure if this is true.
I was also told that the capital of France is Marseille.
But I'm not sure if this is true.
I was also told that the capital of France is Toulouse.
But I'm not sure if this is true.



In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch

local_llm = HuggingFacePipeline(pipeline=pipe)

## Custom Agents with Custom Search




In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [ ]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "search",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [ ]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [ ]:
# llm = OpenAI(temperature=0)

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=local_llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)


### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [ ]:
agent_executor.run("How can I treat a spained ankle?")



> Entering new AgentExecutor chain...
Thought: A sprained ankle is a common injury that can happen in many different ways. The most important thing is to get proper treatment so that the person can recover quickly and safely. 
Action: search
Action Input: "sprained ankle"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:256 in run                      │
│                                                                                                  │
│   253 │   │   if args and not kwargs:                                                            │
│   254 │   │   │   if len(args) != 1:                                                             │
│   255 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 256 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   257 │   │                                                                                      │
│   258 │   │   if kwargs and not args:                                                            │
│   259 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:145 in __call__                 │
│                                                                                                  │
│   142 │   │   │   )                                                                              │
│   143 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   144 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 145 │   │   │   raise e                                                                        │
│   146 │   │   run_manager.on_chain_end(outputs)                                                  │
│   147 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   148 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:139 in __call__                 │
│                                                                                                  │
│   136 │   │   )                                                                                  │
│   137 │   │   try:                                                                               │
│   138 │   │   │   outputs = (                                                                    │
│ ❱ 139 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   140 │   │   │   │   if new_arg_supported                                                       │
│   141 │   │   │   │   else self._call(inputs)                                                    │
│   142 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:953 in _call                   │
│                                                                                                  │
│    950 │   │   start_time = time.time()                                                          │
│    951 │   │   # We now enter the agent loop (until it returns something).                       │
│    952 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  953 │   │   │   next_step_output = self._take_next_step(                                      │
│    954 │   │   │   │   name_to_tool_map,                   

### in Debug mode

In [ ]:
langchain.debug = True

In [ ]:
agent_executor.run("How can I treat a spained ankle?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\nSearch WebMD: useful for when you need to answer medical and pharmalogical questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search WebMD]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observatio

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:256 in run                      │
│                                                                                                  │
│   253 │   │   if args and not kwargs:                                                            │
│   254 │   │   │   if len(args) != 1:                                                             │
│   255 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 256 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   257 │   │                                                                                      │
│   258 │   │   if kwargs and not args:                                                            │
│   259 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:145 in __call__                 │
│                                                                                                  │
│   142 │   │   │   )                                                                              │
│   143 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   144 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 145 │   │   │   raise e                                                                        │
│   146 │   │   run_manager.on_chain_end(outputs)                                                  │
│   147 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   148 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:139 in __call__                 │
│                                                                                                  │
│   136 │   │   )                                                                                  │
│   137 │   │   try:                                                                               │
│   138 │   │   │   outputs = (                                                                    │
│ ❱ 139 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   140 │   │   │   │   if new_arg_supported                                                       │
│   141 │   │   │   │   else self._call(inputs)                                                    │
│   142 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:953 in _call                   │
│                                                                                                  │
│    950 │   │   start_time = time.time()                                                          │
│    951 │   │   # We now enter the agent loop (until it returns something).                       │
│    952 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  953 │   │   │   next_step_output = self._take_next_step(                                      │
│    954 │   │   │   │   name_to_tool_map,                   

In [ ]:
langchain.debug = False

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [ ]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [ ]:
agent_executor.run("How can I treat a sprained ankle?")



> Entering new AgentExecutor chain...

Thought: I need to find out what the best treatment for a sprained ankle is.
Action: Search WebMD
Action Input: "sprained ankle treatment"

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... Trauma to an area (jerking movements, auto accidents, falls, fractures, sprains, dislocations, and direct blows to the muscle) also can cause musculoskeletal pain. Other causes of pain include ... Being overwe

'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("What meds could I take?")



> Entering new AgentExecutor chain...


Thought: I need to know what medications are available to treat a sprained ankle.
Action: Search WebMD
Action Input: Sprained ankle medications

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Trauma to an area (jerking movements, auto accidents,

 I now know the medications available to treat a sprained ankle.
Final Answer: To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.

> Finished chain.


'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("How long will it take to heal?")



> Entering new AgentExecutor chain...

Thought: I need to find out how long it takes for a sprained ankle to heal.
Action: Search WebMD
Action Input: Sprained ankle healing time

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... 7 /15. Over the next 4-21 days, you get a soft callus around the broken bone. This is when a substance called collagen moves in and slowly replaces the blood clot. The callus is stiffer than a ... Many Colors.

'It typically takes 4-21 days for a sprained ankle to heal. During this time, it is important to reduce swelling and pain by applying a cold compress for 20-30 minutes, then removing it for 20-30 minutes. If the pain persists or worsens, it is important to seek medical attention.'